In [ ]:
import pandas as pd
import scipy
from nltk.sentiment.vader import SentimentIntensityAnalyzer;

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv("/data/beer/train.csv")

In [155]:
test = pd.read_csv("/data/beer/test.csv")

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [145]:
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words="english", max_features=1000)
X = vectorizer.fit_transform([str(text) for text in data['review/text']])

In [146]:
X = pd.DataFrame(X.todense())

In [134]:
# sid = SentimentIntensityAnalyzer()
# sentiment = [sid.polarity_scores(str(review))["compound"] for review in data["review/text"]]

# data["user/ageInSeconds"] = data["user/ageInSeconds"].fillna(data['user/ageInSeconds'].mean())
# X["age"] = data['user/ageInSeconds'] / (3600 * 24 * 365)

In [156]:
sid = SentimentIntensityAnalyzer()
test_sentiment = [sid.polarity_scores(str(review))["compound"] for review in test["review/text"]]

In [147]:
X["sentiment"] = sentiment
X["abv"] = data["beer/ABV"]

In [148]:
style = pd.get_dummies(data["beer/style"])
for col in style:
    X["user:" + str(col)] = style[col]

In [108]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = [tokenizer.tokenize(str(review)) for review in data['review/text']]

In [113]:
appearanceWords = ["appearance", "character", "image", "look", "presence", "presentation", "air", "attitude", "bearing", "blind",
              "carriage", "cast", "condition", "countenance", "demeanor", "dress", "expression", "face", "fashion", "feature",
              "figure", "form", "front", "guise", "looks", "manner", "mannerism", "mien", "mode", "outline", "pose", "screen",
              "semblance", "shape", "stamp",'amber','brown','caramel','chocolate','clean','clouded','coffee','creamy','crisp',
              'hazy','head','lacing','orange','orangeamber','pour','red','rich', 'ruby','snifterdeep','solid','tan','toast',
              'vanilla','white']

aromaWords = ["aroma", "bouquet", "odor", "perfume", "scent", "spice", "balm", "fragrance", "incense", "redolence", "smell",
         'burn','candy','coffee','floral','fruity','grapefruity','herbal','lingering','mild','peppery','rich','roasted',
         'sharp','skunky','spice','sweet','toast','vanilla']


palateWords = ["palate", "bias", "partiality", "penchant", "tendency", "attraction", "inclination", "preference", "appetite", 
          "disposition", "gusto", "zest", 'bodied','carbonated','crisp','dense','heavy','kick','lingering','rich','sharp',
          'smooth','soft']

tasteWords = ["taste", "aftertaste", "aroma", "bang", "bitter", "drive", "ginger", "jolt", "kick", "oomph", "palatableness",
         "piquancy", "punch", "relish", "salt", "sapidity", "savor", "savoriness", "smack", "sour", "sting", "suggestion",
         "sweet", "tang", "wallop", "zest", "zing", "zip", "sapor", 'absinthe','aftertaste','biscuity','bitterness','bland',
         'bold','bourbon','burn', 'candy','caramel','carbonated','coffee','creamy','crisp','dense','floral','hop','kick',
         'lingering','malt','mild','peppery','refreshing','rich','roasted','sharp','sour','spice', 'sugar','sweet','tart',
         'tangy','vanilla']

In [115]:
aroma = [0 for _ in sentences]
palate = [0 for _ in sentences]
taste = [0 for _ in sentences]
appearance = [0 for _ in sentences]

i = 0
for paragraph in sentences:
    aromaCount = 0
    palateCount = 0
    tasteCount = 0
    appearanceCount = 0

    for sentence in paragraph:
        if any(a in sentence for a in aromaWords):
            aromaCount += 1
            aroma[i] += sid.polarity_scores(sentence)["compound"]
        if any(a in sentence for a in palateWords):
            palateCount += 1
            palate[i] += sid.polarity_scores(sentence)["compound"]
        if any(a in sentence for a in tasteWords):
            tasteCount += 1
            taste[i] += sid.polarity_scores(sentence)["compound"]
        if any(a in sentence for a in appearanceWords):
            appearanceCount += 1
            appearance[i] += sid.polarity_scores(sentence)["compound"]
    
    if aromaCount > 0:
        aroma[i] /= aromaCount    
    if palateCount > 0:
        palate[i] /= palateCount
    if tasteCount > 0:
        taste[i] /= tasteCount
    if appearanceCount > 0:
        appearance[i] /= appearanceCount
    i += 1

In [158]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
test_sentences = [tokenizer.tokenize(str(review)) for review in test['review/text']]

In [164]:
test_aroma = [0 for _ in test_sentences]
test_palate = [0 for _ in test_sentences]
test_taste = [0 for _ in test_sentences]
test_appearance = [0 for _ in test_sentences]

i = 0
for paragraph in test_sentences:
    aromaCount = 0
    palateCount = 0
    tasteCount = 0
    appearanceCount = 0

    for sentence in paragraph:
        if any(a in sentence for a in aromaWords):
            aromaCount += 1
            test_aroma[i] += sid.polarity_scores(sentence)["compound"]
        if any(a in sentence for a in palateWords):
            palateCount += 1
            test_palate[i] += sid.polarity_scores(sentence)["compound"]
        if any(a in sentence for a in tasteWords):
            tasteCount += 1
            test_taste[i] += sid.polarity_scores(sentence)["compound"]
        if any(a in sentence for a in appearanceWords):
            appearanceCount += 1
            test_appearance[i] += sid.polarity_scores(sentence)["compound"]
    
    if aromaCount > 0:
        test_aroma[i] /= aromaCount    
    if palateCount > 0:
        test_palate[i] /= palateCount
    if tasteCount > 0:
        test_taste[i] /= tasteCount
    if appearanceCount > 0:
        test_appearance[i] /= appearanceCount
    i += 1

In [129]:
X

<37500x1097 sparse matrix of type '<class 'numpy.float64'>'
	with 1891107 stored elements in Compressed Sparse Row format>

In [106]:
data.columns

Index(['index', 'beer/ABV', 'beer/beerId', 'beer/brewerId', 'beer/name',
       'beer/style', 'review/appearance', 'review/aroma', 'review/overall',
       'review/palate', 'review/taste', 'review/text', 'review/timeStruct',
       'review/timeUnix', 'user/ageInSeconds', 'user/birthdayRaw',
       'user/birthdayUnix', 'user/gender', 'user/profileName'],
      dtype='object')

In [192]:
sent = {"palate" : palate, "aroma" : aroma, "taste" : taste, "appearance" : appearance}
test_sent = {"palate" : test_palate, "aroma" : test_aroma, "taste" : test_taste, "appearance" : test_appearance}
reviews = ["appearance", "aroma", "overall", "palate", "taste"]

diff = list(set(pd.get_dummies(data["beer/style"]).columns) - set(pd.get_dummies(test["beer/style"]).columns))

predictions = pd.DataFrame(index = test["index"])

for review in reviews:
    model = LinearRegression()
    
    vectorizer = CountVectorizer(ngram_range=(1,2), stop_words="english", max_features=1000)
    X = vectorizer.fit_transform([str(text) for text in data['review/text']])

    X = pd.DataFrame(X.todense())

    X["sentiment"] = sentiment
    X["abv"] = data["beer/ABV"]

    style = pd.get_dummies(data["beer/style"])
    for col in style:
        X["style:" + str(col)] = style[col]
        
    if review != "overall":            
        X[review] = sent[review]

    X.columns = X.columns.astype(str)
    X.sort_index(axis=1, inplace=True)
    print(X.head())
        
#     X = scipy.sparse.csr_matrix(X.values)
#     print(review, -cross_val_score(model, X, data["review/" + review], cv=10, scoring="neg_mean_squared_error").mean())

    model.fit(X, data["review/"+review])
    
    Xtest = vectorizer.transform([str(text) for text in test['review/text']])
    Xtest = pd.DataFrame(Xtest.todense())

    Xtest["sentiment"] = test_sentiment
    Xtest["abv"] = test["beer/ABV"]

    style = pd.get_dummies(test["beer/style"])
    for col in style:
        Xtest["style:" + str(col)] = style[col]
        
    for col in diff:
        Xtest["style:" + col] = [0 for _ in Xtest["abv"]]
        
    if review != "overall":            
        Xtest[review] = test_sent[review]
        
    Xtest.columns = Xtest.columns.astype(str)
    Xtest.sort_index(axis=1, inplace=True)
    print(Xtest.head())

#     Xtest = scipy.sparse.csr_matrix(Xtest.values)
    predictions["review/"+review] = model.predict(Xtest)

   0  1  10  100  101  102  103  104  105  106      ...        \
0  0  0   0    0    0    0    0    0    0    0      ...         
1  0  0   0    0    0    0    1    0    0    0      ...         
2  0  0   0    0    0    0    1    0    0    0      ...         
3  0  1   0    0    0    0    0    0    0    0      ...         
4  0  0   0    0    1    0    0    0    0    0      ...         

   style:Scotch Ale / Wee Heavy  style:Scottish Ale  \
0                             0                   0   
1                             0                   0   
2                             0                   0   
3                             0                   0   
4                             0                   0   

   style:Scottish Gruit / Ancient Herbed Ale  style:Smoked Beer  style:Tripel  \
0                                          0                  0             0   
1                                          0                  0             0   
2                                  

In [193]:
predictions.to_csv("output.csv")

review/palate:     0.266360520049
review/aroma:      0.27499270306
review/taste:      0.277464650465
review/appearance: 0.236480775284
review/overall:    0.310799823099

review/palate:     0.253683886305
review/aroma:      0.253140656648
review/taste:      0.259079936169
review/appearance: 0.2215648635
review/overall:    0.298516358874

review/palate:     0.253673400152
review/aroma:      0.253174497474
review/taste:      0.259068148975
review/appearance: 0.221632953296
review/overall:    0.298497046681

review/palate:     0.253498607764
review/aroma:      0.251732535731
review/taste:      0.257598991441
review/appearance: 0.219973970342
review/overall:    0.298497046681